In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    FFF_Train_Decile3 = pd.read_pickle("FFF_Train_Decile3.pkl")
    FFF_Train_Targets_1D = pd.read_pickle("FFF_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("FFF_Test_Decile3.pkl")
    y_test = pd.read_pickle("FFF_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(FFF_Train_Decile3, FFF_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150, 200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM FFF Decile 3')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3055/3055 [==============================] - 3s 832us/step - loss: 0.4741 - acc: 0.7895 - val_loss: 0.4456 - val_acc: 0.8024
Epoch 38/100
3055/3055 [==============================] - 3s 843us/step - loss: 0.4764 - acc: 0.7918 - val_loss: 0.4444 - val_acc: 0.7958
Epoch 39/100
3055/3055 [==============================] - 3s 878us/step - loss: 0.4758 - acc: 0.7899 - val_loss: 0.4500 - val_acc: 0.7971
Epoch 40/100
3055/3055 [==============================] - 3s 854us/step - loss: 0.4731 - acc: 0.7866 - val_loss: 0.4454 - val_acc: 0.7984
Epoch 41/100
3055/3055 [==============================] - 3s 842us/step - loss: 0.4711 - acc: 0.7915 - val_loss: 0.4417 - val_acc: 0.7971
Epoch 42/100
3055/3055 [==============================] - 3s 838us/step - loss: 0.4731 - acc: 0.7859 - val_loss: 0.4433 - val_acc: 0.8050
Epoch 43/100
3055/3055 [==============================] - 3s 861us/step - loss: 0.4713 - acc: 0.7902 - val_loss: 0.4414 - val_acc: 0.8010
Epoch 44/100
3055/3055 [=======================

Epoch 96/100
3055/3055 [==============================] - 3s 841us/step - loss: 0.4606 - acc: 0.7866 - val_loss: 0.4380 - val_acc: 0.8024
Epoch 97/100
3055/3055 [==============================] - 3s 870us/step - loss: 0.4639 - acc: 0.7912 - val_loss: 0.4376 - val_acc: 0.8024
Epoch 98/100
3055/3055 [==============================] - 3s 919us/step - loss: 0.4613 - acc: 0.7889 - val_loss: 0.4369 - val_acc: 0.8037
Epoch 99/100
3055/3055 [==============================] - 3s 891us/step - loss: 0.4615 - acc: 0.7921 - val_loss: 0.4373 - val_acc: 0.8024
Epoch 100/100
3055/3055 [==============================] - 3s 879us/step - loss: 0.4656 - acc: 0.7899 - val_loss: 0.4376 - val_acc: 0.8010
Test accuracy: 0.8010471201067819
Train on 3055 samples, validate on 764 samples
Epoch 1/150
3055/3055 [==============================] - 4s 1ms/step - loss: 0.6187 - acc: 0.6439 - val_loss: 0.5390 - val_acc: 0.7251
Epoch 2/150
3055/3055 [==============================] - 3s 901us/step - loss: 0.5300 - acc: 

3055/3055 [==============================] - 3s 868us/step - loss: 0.4544 - acc: 0.7915 - val_loss: 0.4265 - val_acc: 0.8037
Epoch 54/150
3055/3055 [==============================] - 3s 849us/step - loss: 0.4548 - acc: 0.7964 - val_loss: 0.4265 - val_acc: 0.8037
Epoch 55/150
3055/3055 [==============================] - 3s 849us/step - loss: 0.4493 - acc: 0.8000 - val_loss: 0.4265 - val_acc: 0.8037
Epoch 56/150
3055/3055 [==============================] - 3s 879us/step - loss: 0.4514 - acc: 0.7918 - val_loss: 0.4265 - val_acc: 0.8037
Epoch 57/150
3055/3055 [==============================] - 3s 890us/step - loss: 0.4502 - acc: 0.7918 - val_loss: 0.4264 - val_acc: 0.8037
Epoch 58/150
3055/3055 [==============================] - 3s 892us/step - loss: 0.4507 - acc: 0.7948 - val_loss: 0.4263 - val_acc: 0.8037
Epoch 59/150
3055/3055 [==============================] - 3s 858us/step - loss: 0.4496 - acc: 0.7957 - val_loss: 0.4262 - val_acc: 0.8037
Epoch 60/150
3055/3055 [=======================

3055/3055 [==============================] - 2s 803us/step - loss: 0.4479 - acc: 0.7967 - val_loss: 0.4263 - val_acc: 0.8063
Epoch 113/150
3055/3055 [==============================] - 2s 797us/step - loss: 0.4507 - acc: 0.7928 - val_loss: 0.4265 - val_acc: 0.8076
Epoch 114/150
3055/3055 [==============================] - 2s 811us/step - loss: 0.4501 - acc: 0.7964 - val_loss: 0.4264 - val_acc: 0.8063
Epoch 115/150
3055/3055 [==============================] - 2s 793us/step - loss: 0.4488 - acc: 0.7974 - val_loss: 0.4263 - val_acc: 0.8076
Epoch 116/150
3055/3055 [==============================] - 2s 799us/step - loss: 0.4468 - acc: 0.7974 - val_loss: 0.4263 - val_acc: 0.8063
Epoch 117/150
3055/3055 [==============================] - 3s 818us/step - loss: 0.4477 - acc: 0.7948 - val_loss: 0.4263 - val_acc: 0.8063
Epoch 118/150
3055/3055 [==============================] - 3s 970us/step - loss: 0.4492 - acc: 0.7954 - val_loss: 0.4262 - val_acc: 0.8063
Epoch 119/150
3055/3055 [================

...

Epoch 8/200
3055/3055 [==============================] - 3s 861us/step - loss: 0.4772 - acc: 0.7908 - val_loss: 0.4376 - val_acc: 0.7932
Epoch 9/200
3055/3055 [==============================] - 3s 887us/step - loss: 0.4688 - acc: 0.7827 - val_loss: 0.4336 - val_acc: 0.7971
Epoch 10/200
3055/3055 [==============================] - 3s 857us/step - loss: 0.4637 - acc: 0.7853 - val_loss: 0.4491 - val_acc: 0.7906
Epoch 11/200
3055/3055 [==============================] - 3s 884us/step - loss: 0.4642 - acc: 0.7915 - val_loss: 0.4497 - val_acc: 0.7853
Epoch 12/200
3055/3055 [==============================] - 3s 858us/step - loss: 0.4611 - acc: 0.7918 - val_loss: 0.4356 - val_acc: 0.7932
Epoch 13/200
3055/3055 [==============================] - 3s 884us/step - loss: 0.4651 - acc: 0.7902 - val_loss: 0.4324 - val_acc: 0.7971
Epoch 14/200
3055/3055 [==============================] - 3s 877us/step - loss: 0.4618 - acc: 0.7853 - val_loss: 0.4370 - val_acc: 0.7906
Epoch 15/200
3055/3055 [============

Epoch 66/200
3055/3055 [==============================] - 3s 910us/step - loss: 0.4388 - acc: 0.8052 - val_loss: 0.4135 - val_acc: 0.8154
Epoch 67/200
3055/3055 [==============================] - 3s 929us/step - loss: 0.4370 - acc: 0.8010 - val_loss: 0.4135 - val_acc: 0.8154
Epoch 68/200
3055/3055 [==============================] - 3s 839us/step - loss: 0.4341 - acc: 0.8016 - val_loss: 0.4135 - val_acc: 0.8154
Epoch 69/200
3055/3055 [==============================] - 3s 922us/step - loss: 0.4342 - acc: 0.7987 - val_loss: 0.4135 - val_acc: 0.8141
Epoch 70/200
3055/3055 [==============================] - 3s 882us/step - loss: 0.4348 - acc: 0.8020 - val_loss: 0.4135 - val_acc: 0.8141
Epoch 71/200
3055/3055 [==============================] - 3s 927us/step - loss: 0.4363 - acc: 0.8016 - val_loss: 0.4136 - val_acc: 0.8141
Epoch 72/200
3055/3055 [==============================] - 3s 1ms/step - loss: 0.4406 - acc: 0.8020 - val_loss: 0.4137 - val_acc: 0.8141
Epoch 73/200
3055/3055 [============

3055/3055 [==============================] - 3s 874us/step - loss: 0.4341 - acc: 0.7974 - val_loss: 0.4138 - val_acc: 0.8154
Epoch 126/200
3055/3055 [==============================] - 3s 925us/step - loss: 0.4341 - acc: 0.8036 - val_loss: 0.4137 - val_acc: 0.8154
Epoch 127/200
3055/3055 [==============================] - 3s 898us/step - loss: 0.4310 - acc: 0.8007 - val_loss: 0.4136 - val_acc: 0.8154
Epoch 128/200
3055/3055 [==============================] - 3s 874us/step - loss: 0.4321 - acc: 0.8043 - val_loss: 0.4136 - val_acc: 0.8154
Epoch 129/200
3055/3055 [==============================] - 3s 893us/step - loss: 0.4355 - acc: 0.7984 - val_loss: 0.4136 - val_acc: 0.8154
Epoch 130/200
3055/3055 [==============================] - 3s 880us/step - loss: 0.4351 - acc: 0.8026 - val_loss: 0.4136 - val_acc: 0.8154
Epoch 131/200
3055/3055 [==============================] - 3s 912us/step - loss: 0.4300 - acc: 0.8049 - val_loss: 0.4137 - val_acc: 0.8154
Epoch 132/200
3055/3055 [================

3055/3055 [==============================] - 3s 861us/step - loss: 0.4366 - acc: 0.8036 - val_loss: 0.4138 - val_acc: 0.8168
Epoch 185/200
3055/3055 [==============================] - 3s 859us/step - loss: 0.4276 - acc: 0.8029 - val_loss: 0.4138 - val_acc: 0.8181
Epoch 186/200
3055/3055 [==============================] - 3s 896us/step - loss: 0.4332 - acc: 0.8003 - val_loss: 0.4136 - val_acc: 0.8141
Epoch 187/200
3055/3055 [==============================] - 3s 862us/step - loss: 0.4336 - acc: 0.8065 - val_loss: 0.4137 - val_acc: 0.8154
Epoch 188/200
3055/3055 [==============================] - 3s 859us/step - loss: 0.4330 - acc: 0.8059 - val_loss: 0.4137 - val_acc: 0.8181
Epoch 189/200
3055/3055 [==============================] - 3s 896us/step - loss: 0.4356 - acc: 0.8046 - val_loss: 0.4137 - val_acc: 0.8181
Epoch 190/200
3055/3055 [==============================] - 3s 874us/step - loss: 0.4300 - acc: 0.8059 - val_loss: 0.4138 - val_acc: 0.8181
Epoch 191/200
3055/3055 [================

3055/3055 [==============================] - 3s 837us/step - loss: 0.4484 - acc: 0.7954 - val_loss: 0.4271 - val_acc: 0.8037
Epoch 43/50
3055/3055 [==============================] - 3s 842us/step - loss: 0.4494 - acc: 0.7987 - val_loss: 0.4270 - val_acc: 0.8050
Epoch 44/50
3055/3055 [==============================] - 3s 845us/step - loss: 0.4417 - acc: 0.7935 - val_loss: 0.4272 - val_acc: 0.8037
Epoch 45/50
3055/3055 [==============================] - 3s 846us/step - loss: 0.4459 - acc: 0.7921 - val_loss: 0.4272 - val_acc: 0.8037
Epoch 46/50
3055/3055 [==============================] - 3s 836us/step - loss: 0.4489 - acc: 0.7902 - val_loss: 0.4272 - val_acc: 0.8037
Epoch 47/50
3055/3055 [==============================] - 3s 831us/step - loss: 0.4506 - acc: 0.7925 - val_loss: 0.4270 - val_acc: 0.8037
Epoch 48/50
3055/3055 [==============================] - 3s 863us/step - loss: 0.4429 - acc: 0.7967 - val_loss: 0.4269 - val_acc: 0.8037
Epoch 49/50
3055/3055 [==============================

In [3]:
0.93484

0.93484

In [ ]:
#0.84